In [2]:
%pip install pandas snowflake-connector-python snowflake-connector-python[pandas] kaggle --quiet
print("Packages installed")


Note: you may need to restart the kernel to use updated packages.
Packages installed


In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
# Kaggle auth & download

import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

# my kaggle creds
kaggle_dict = {
    "username": os.getenv("KAGGLE_USERNAME"),  # put your username
    "key": os.getenv("KAGGLE_KEY")  # put your API key
}

# make kaggle folder in home
kaggle_dir = os.path.expanduser("~/kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

# path for kaggle.json
kaggle_json_path = os.path.join(kaggle_dir, "kaggle.json")

# write kaggle.json
with open(kaggle_json_path, "w", encoding="utf-8") as f:
    json.dump(kaggle_dict, f, indent=4)

# set file permissions
os.chmod(kaggle_json_path, 0o600)

# set env var
os.environ["KAGGLE_CONFIG_DIR"] = kaggle_dir

# restart the kernel now so API sees the config

# import kaggle api
api = KaggleApi()

# authenticate
api.authenticate()
print("Kaggle authentication successful")

# download the dataset
dataset = "zynicide/wine-reviews"  # dataset slug
download_dir = os.path.expanduser("~/kaggle_downloads")
os.makedirs(download_dir, exist_ok=True)

api.dataset_download_files(dataset, path=download_dir, unzip=True)
print("Kaggle dataset downloaded and unzipped to", download_dir)

# check files
print("Files in download folder:", os.listdir(download_dir))


Kaggle authentication successful
Dataset URL: https://www.kaggle.com/datasets/zynicide/wine-reviews
Kaggle dataset downloaded and unzipped to /home/developer/kaggle_downloads
Files in download folder: ['winemag-data-130k-v2.csv', 'winemag-data-130k-v2.json', 'winemag-data_first150k.csv']


In [7]:
# Load & clean the Kaggle dataset

import pandas as pd
import os

# path to the CSV we want
file_path = os.path.join(os.path.expanduser("~/kaggle_downloads"), "winemag-data_first150k.csv")

# load CSV into a DataFrame
df = pd.read_csv(file_path)
print("Data loaded. Shape:", df.shape)

# drop rows where essential columns are missing
essential_cols = ['country', 'description', 'points', 'price', 'variety', 'winery']
df = df.dropna(subset=essential_cols)

# convert numeric columns to proper type
df['points'] = df['points'].astype(int)
df['price'] = df['price'].astype(float)

# strip spaces from text columns
text_cols = ['country', 'description', 'designation', 'province', 'region_1', 'region_2', 'variety', 'winery']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].str.strip()

# reset index after cleaning
df = df.reset_index(drop=True)

print("Data cleaned. Shape:", df.shape)

# show first 5 rows to verify
df.head()

Data loaded. Shape: (150930, 11)
Data cleaned. Shape: (137230, 11)


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [8]:
# Upload cleaned data to Snowflake (full working version)

import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

# connect to Snowflake
conn = snowflake.connector.connect(
    user = os.getenv("SNOWFLAKE_USER"),                     # your Snowflake username
    password = os.getenv("SNOWFLAKE_PASSWORD"),        # your Snowflake password
    account='we67129.me-central2.gcp',# your Snowflake account from trial URL
    role='SYSADMIN'                    # optional
)
print("Snowflake connection successful")

# create database if not exists
conn.cursor().execute("CREATE DATABASE IF NOT EXISTS KAGGLE_ETL")
conn.cursor().execute("USE DATABASE KAGGLE_ETL")

# create schema if not exists
conn.cursor().execute("CREATE SCHEMA IF NOT EXISTS WINE_SCHEMA")
conn.cursor().execute("USE SCHEMA WINE_SCHEMA")

# table name in Snowflake
table_name = "WINE_REVIEWS"

# fix DataFrame column names to be Snowflake-friendly
df.columns = [c.upper().replace(" ", "_").replace("-", "_") for c in df.columns]

# upload DataFrame to Snowflake, auto-create table if not exists
success, nchunks, nrows, _ = write_pandas(conn, df, table_name, auto_create_table=True)

if success:
    print(f"Data uploaded to Snowflake table '{table_name}'. Rows: {nrows}")
else:
    print("Upload failed")

# close connection
conn.close()

Snowflake connection successful
Data uploaded to Snowflake table 'WINE_REVIEWS'. Rows: 137230
